In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

# cogvlm2-llama3-caption

In [2]:
import io

import argparse
import numpy as np
import torch
from decord import cpu, VideoReader, bridge
from transformers import AutoModelForCausalLM, AutoTokenizer

from matplotlib import pyplot as plt
import pandas as pd

from tqdm import tqdm
import time

/root/anaconda3/envs/cogvideo_caption/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/anaconda3/envs/cogvideo_caption/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
MODEL_PATH = "THUDM/cogvlm2-llama3-caption"

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[
    0] >= 8 else torch.float16

In [4]:

parser = argparse.ArgumentParser(description="CogVLM2-Video CLI Demo")
parser.add_argument('--quant', type=int, choices=[4, 8], help='Enable 4-bit or 8-bit precision loading', default=0)
# args = parser.parse_args([])
args = parser.parse_args([])

In [5]:
def load_video(video_data, strategy='chat'):
    bridge.set_bridge('torch')
    mp4_stream = video_data
    num_frames = 24
    decord_vr = VideoReader(io.BytesIO(mp4_stream), ctx=cpu(0))

    frame_id_list = None
    total_frames = len(decord_vr)
    if strategy == 'base':
        clip_end_sec = 60
        clip_start_sec = 0
        start_frame = int(clip_start_sec * decord_vr.get_avg_fps())
        end_frame = min(total_frames,
                        int(clip_end_sec * decord_vr.get_avg_fps())) if clip_end_sec is not None else total_frames
        frame_id_list = np.linspace(start_frame, end_frame - 1, num_frames, dtype=int)
    elif strategy == 'chat':
        timestamps = decord_vr.get_frame_timestamp(np.arange(total_frames))
        timestamps = [i[0] for i in timestamps]
        max_second = round(max(timestamps)) + 1
        frame_id_list = []
        for second in range(max_second):
            closest_num = min(timestamps, key=lambda x: abs(x - second))
            index = timestamps.index(closest_num)
            frame_id_list.append(index)
            if len(frame_id_list) >= num_frames:
                break

    video_data = decord_vr.get_batch(frame_id_list)
    video_data = video_data.permute(3, 0, 1, 2)
    return video_data

In [6]:
def get_video_length_in_seconds(video_data):
      bridge.set_bridge('torch')
      mp4_stream = video_data
      num_frames = 24
      decord_vr = VideoReader(io.BytesIO(mp4_stream), ctx=cpu(0))

      frame_id_list = None
      total_frames = len(decord_vr)

      timestamps = decord_vr.get_frame_timestamp(np.arange(total_frames))
      timestamps = [i[0] for i in timestamps]
      max_second = round(max(timestamps)) + 1

      return max_second

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=TORCH_TYPE,
    trust_remote_code=True,
    device_map = "auto"
).eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
A new version of the following files was downloaded from https://huggingface.co/THUDM/cogvlm2-llama3-caption:
- configuration_cogvlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/cogvlm2-llama3-caption:
- util.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/cogvlm2-llama3-caption:
- visual.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded fr

In [8]:
def predict(prompt, video_data, temperature):
    strategy = 'chat'

    video = load_video(video_data, strategy=strategy)

    print(video.size())

    history = []
    query = prompt
    inputs = model.build_conversation_input_ids(
        tokenizer=tokenizer,
        query=query,
        images=[video],
        history=history,
        template_version=strategy
    )
    # inputs = {
    #     'input_ids': inputs['input_ids'].unsqueeze(0).to('cuda'),
    #     'token_type_ids': inputs['token_type_ids'].unsqueeze(0).to('cuda'),
    #     'attention_mask': inputs['attention_mask'].unsqueeze(0).to('cuda'),
    #     'images': [[inputs['images'][0].to('cuda').to(TORCH_TYPE)]],
    # }
    inputs = {
        'input_ids': inputs['input_ids'].unsqueeze(0).cuda(),
        'token_type_ids': inputs['token_type_ids'].unsqueeze(0).cuda(),
        'attention_mask': inputs['attention_mask'].unsqueeze(0).cuda(),
        'images': [[inputs['images'][0].cuda().to(TORCH_TYPE)]],
    }
    gen_kwargs = {
        "max_new_tokens": 500,
        "pad_token_id": 128002,
        "top_k": 1,
        "do_sample": False,
        "top_p": 0.1,
        "temperature": temperature,
    }
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

# generate caption for a dataset

In [11]:
PROMPT="Please describe this video in detail." # original prompt

### 09/20/2024
## general prompt
# with examples
# PROMPT="A video is given by providing three frames in chronological order. Describe this video and its style to generate a description. Pay attention to all objects in the video. Do not describe each frame individually. Do not reply with words like 'first frame'. The description should be useful for AI to re-generate the video. The description should be no more than 140 words. Here are some examples of good descriptions: 1. A stylish woman in a black leather jacket, long flowing red dress, black boots, and dark sunglasses strides confidently down a bustling, neon-lit Tokyo street. Her bold red lipstick complements her outfit, while she holds a sleek black purse in one hand. The damp pavement glistens, reflecting the vibrant neon signs from surrounding buildings, casting a kaleidoscope of colors across the scene. Pedestrians move around her, but her confident walk makes her stand out amidst the crowd. The camera follows her from a slightly low angle, emphasizing her commanding presence, while the shallow depth of field blurs the surrounding faces, keeping her in sharp focus. The bright neon lights contrast with the dark tones of her attire, enhancing the scene’s urban, futuristic aesthetic. The mood is electric and stylish, blending modern fashion with the vibrant, fast-paced energy of Tokyo’s nightlife. 2. Several massive wooly mammoths slowly make their way across a snowy meadow, their thick fur rippling in the gusty wind. The towering creatures are framed by a landscape of snow-covered trees and imposing, jagged mountains in the distance. Above, wispy clouds stretch across the sky, with the mid-afternoon sun casting a soft, warm glow over the scene, enhancing the contrast between the cold snow and the rich textures of the mammoths' fur. The low camera angle captures the scale of the mammoths, making them appear even more majestic as they dominate the foreground. Their slow, steady movements create a peaceful yet powerful atmosphere. The warm sunlight filtering through the clouds adds a golden hue to the snowy expanse, while the dramatic mountains provide a rugged backdrop. The combination of lighting, natural beauty, and the sheer size of the mammoths creates a timeless and awe-inspiring visual. 3. A drone glides over the rugged cliffs of Big Sur’s Garay Point Beach, capturing the raw power of waves crashing against the jagged coastline. The deep blue waters, capped with white-tipped waves, shimmer under the warm glow of the setting sun. The drone's high vantage point reveals the steep drop from the coastal road to the beach below, emphasizing the dramatic vertical landscape. Lush green shrubbery clings to the cliffs, adding texture and color to the rocky terrain. In the distance, a small island with a lighthouse stands against the horizon, its isolated presence enhancing the sense of vastness and tranquility. The composition masterfully balances the rugged cliffs, turbulent waters, and serene island, while the golden sunlight casts long shadows, adding depth and warmth. The dynamic movement of the ocean and the sweeping coastal views create a striking visual of nature’s untamed beauty."
# without examples
# PROMPT="A video is given by providing several frames in chronological order. Describe this video and its style to generate a description. Pay attention to all objects in the video. Do not describe each frame individually. Do not reply with words like 'first frame'. The description should be useful for AI to re-generate the video. The description should be no more than 140 words."

## film prompt
# with examples
# PROMPT="A video is given by providing three frames in chronological order. Describe this video including the objects, their performance, the scene, and the visual style. Pay attention to all the objects in the video. Besides, describe the video based on the following essential elements of professional film footages: 1. Camera: Camera movement, lens, and depth of field directly affect the quality, style,and narrative of the footage. 2. Lighting: Lighting direction and color tone are crucial for setting the mood of the scene. It affects how the colors and shadows appear and can be used creatively to influence the story's atmosphere and the audience's perception of characters and settings. 3. Composition: This involves the arrangement of elements within the frame, including characters, props, and scenery. Composition techniques, such as the rule of thirds, leading lines, and framing, can help guide the viewer’s attention and make the visuals more engaging. Note: Do not describe each frame individually. Do not reply with words like 'first frame'. The entire description should be no more than 140 words, which should be useful for AI to re-generate the video. Here are some examples of descriptions: 1. A stylish woman in a black leather jacket and vibrant red dress walks confidently through a neon-lit Tokyo street, her figure dominating the frame as the camera follows her with a shallow depth of field. The busy urban scene blurs softly behind her, highlighting her bold presence amidst a sea of lights and reflections from the rain-slicked pavement. The camera's steady movement keeps her at the center, guiding the viewer’s focus on her assertive stride. Dynamic lighting intensifies the colors, with neon hues bouncing off the wet ground, adding depth and contrast to the visuals. Clever use of the rule of thirds draws attention to the woman while maintaining balance with the bustling background. The lighting enhances the mood, bathing the scene in vibrant colors, while the shallow focus emphasizes her isolation amidst the crowd, heightening the drama and sense of purpose in her walk. 2. Several wooly mammoths move steadily across a vast snowy meadow, their thick fur rippling in the brisk wind. A low-angle camera captures the scene, giving the mammoths a towering, majestic presence as they traverse the rugged landscape. The deep focus keeps both the mammoths and the snow-laden trees in sharp detail, while the dramatic mountains in the background add grandeur to the composition. Mid-afternoon sunlight casts a golden glow over the scene, creating a striking contrast between the warm light and the cold snow. The lighting highlights the texture of the mammoths' fur, making them stand out against the icy surroundings. The composition artfully balances these creatures against the dramatic backdrop, using leading lines from the terrain to guide the viewer’s eyes. This harmony of camera movement, light, and natural elements enhances the narrative, conveying the mammoths' dominance and resilience in a harsh environment. 3. A drone soars over Big Sur’s Garay Point Beach, capturing powerful waves as they crash against the steep, jagged cliffs. The camera’s aerial perspective emphasizes the vastness of the coastline, revealing the steep drop to the rocky shore below. In the distance, a small island with a lighthouse adds a focal point to the scene, enhancing the visual narrative of isolation and rugged beauty. The setting sun bathes the landscape in a warm, golden light, contrasting beautifully with the deep blue waters and white-tipped waves. The soft, natural lighting accentuates the textures of the cliffs and the movement of the ocean, adding depth and drama to the composition. The rule of thirds is expertly used, with the cliffs occupying one side, the ocean in the middle, and the distant island drawing the eye outward, creating a balanced and visually captivating scene."
# without examples
# PROMPT="A video is given by providing several frames in chronological order. Describe this video including the objects, their performance, the scene, and the visual style. Pay attention to all the objects in the video. Besides, describe the video based on the following essential elements of professional film footages: 1. Camera: Camera movement, lens, and depth of field directly affect the quality, style,and narrative of the footage. 2. Lighting: Lighting direction and color tone are crucial for setting the mood of the scene. It affects how the colors and shadows appear and can be used creatively to influence the story's atmosphere and the audience's perception of characters and settings. 3. Composition: This involves the arrangement of elements within the frame, including characters, props, and scenery. Composition techniques, such as the rule of thirds, leading lines, and framing, can help guide the viewer’s attention and make the visuals more engaging. Note: Do not describe each frame individually. Do not reply with words like 'first frame'. The entire description should be no more than 140 words, which should be useful for AI to re-generate the video."

video_csv_path = "/root/projects/Data/CelebV-HQ/video_paths_subset_100_full_path.csv" # provide the path here
# new_csv_path = video_csv_path.replace(".csv", "_caption_CogVLM2Caption_GeneralPromptNoExamples_140words_09_20.csv") # change the name when you change the prompt!
# new_csv_path = video_csv_path.replace(".csv", "_caption_CogVLM2Caption_OriginalPrompt.csv") # change the name when you change the prompt!

In [12]:
df = pd.read_csv(video_csv_path)
video_path_list = df["path"].tolist()
caption_list = []

video_seconds_list = []
process_time_list = []

for i, video_path in enumerate(tqdm(video_path_list)):
      temperature = 0.1

      start_time = time.time()
      video_data = open(video_path, 'rb').read()
      response = predict(PROMPT, video_data, temperature)
      end_time = time.time()
      
      caption_list.append(response)

      # statistics
      process_time_list.append(end_time - start_time)
      max_seconds = get_video_length_in_seconds(video_data)
      video_seconds_list.append(max_seconds)

      print(f"{video_path} {max_seconds} seconds\n{response}\n\n")

      if i == 30:
            break

# save
# df["text"] = caption_list
# df.to_csv(new_csv_path, index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([3, 6, 512, 512])


/root/anaconda3/envs/cogvideo_caption/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/anaconda3/envs/cogvideo_caption/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/root/anaconda3/envs/cogvideo_caption/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:562: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  1%|          | 1/10

/root/projects/Data/CelebV-HQ/celebvhq/-mjnbKL7fHQ_2.mp4 6 seconds
 A muscular Black man with short hair and a goatee, wearing a white sleeveless top, stands in an indoor gym, exuding determination and focus. The gym is equipped with a clock and a bulletin board, suggesting a setting of physical activity or training. As time passes, the man maintains his intense gaze and posture, indicating a serious conversation or confrontation with another person, who is not fully visible. The gym's atmosphere is highlighted by the presence of a clock and a poster, adding to the scene's intensity. The man's muscular build and the gym's equipment suggest a narrative of physical activity or training.


torch.Size([3, 14, 512, 512])


/root/anaconda3/envs/cogvideo_caption/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/anaconda3/envs/cogvideo_caption/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/root/anaconda3/envs/cogvideo_caption/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:562: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  2%|▏         | 2/10

...

/root/projects/Data/CelebV-HQ/celebvhq/GcB7m_3Tlv8_1.mp4 4 seconds
A man with short, light brown hair and stubble is standing in a dimly lit room, wearing a black jacket over a white shirt. He appears to be in a serious conversation or interview, looking off-camera with an earnest expression. The room's wooden ceiling and beams add to the intimate atmosphere. As time passes, the man's expression shifts to one of deep thought or concern, with his eyes slightly narrowed and brows furrowed. The warm lighting continues to highlight his features against the dark background, maintaining the video's intimate and contemplative mood.


torch.Size([3, 4, 512, 512])


 30%|███       | 30/100 [06:11<12:51, 11.03s/it]

/root/projects/Data/CelebV-HQ/celebvhq/GplPjaD4VYk_0.mp4 4 seconds
A woman with dark hair tied up in an elegant bun and wearing a white shirt under a black vest is seen in a professional setting, possibly a kitchen or server area, with a blurred background featuring posters and a framed picture. Her expression is one of concentration and determination, suggesting she is engaged in a serious conversation or task. The scene is from a television show or movie, as indicated by the text "billibili" in the upper right corner. The same woman appears again, maintaining her focused demeanor in what seems to be a formal or professional environment.


torch.Size([3, 8, 512, 512])


 30%|███       | 30/100 [06:22<14:53, 12.77s/it]

/root/projects/Data/CelebV-HQ/celebvhq/H9gqgAcOaE4_0.mp4 8 seconds
 A man in historical attire, including a white shirt and dark vest, is seen in various states of shock and distress against a stone wall. Initially, he appears with his mouth open and eyes wide, suggesting a moment of intense emotion. As moments pass, his expressions evolve from shock to fear, with his eyes wide and mouth agape, indicating a reaction to an unforeseen event. The dim lighting throughout these scenes casts dramatic shadows, enhancing the tension and mystery. The presence of the text 'A24' suggests a connection to a film or television production.




In [13]:
video_seconds_list = np.array(video_seconds_list)
process_time_list = np.array(process_time_list)

# print(f'processing time per video seconds:{np.mean(process_time_list/video_seconds_list)}')

print(f'processing time per video seconds:{process_time_list.sum()/video_seconds_list.sum()}')

print(f'total video time:{process_time_list.sum()}')

processing time per video seconds:2.159343232287473
processing time per video seconds:1.8137862784037657


In [1]:
6000000 * 10 * 1.5 / 3600 / 24

1041.6666666666667